## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pypfopt.plotting as plotting
from pypfopt.expected_returns import mean_historical_return
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import discrete_allocation
from pypfopt import expected_returns
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import objective_functions
from pypfopt import base_optimizer
from pypfopt.discrete_allocation import DiscreteAllocation
from pypfopt.hierarchical_portfolio import HRPOpt

from custom_allocation import CustomDiscreteAllocation

In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page containing the S&P 500 components
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Send a GET request to the URL and get the page content
response = requests.get(url)
content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Find the table containing the stock data
table = soup.find('table', {'class': 'wikitable sortable'})

# Find all rows in the table
rows = table.find_all('tr')

# Iterate over the rows and extract the stock symbols
sp500_stocks = []
for row in rows[1:]:
    cells = row.find_all('td')
    stock_symbol = cells[0].text.strip()
    sp500_stocks.append(stock_symbol)

# Print the list of S&P 500 stocks
for stock in sp500_stocks:
    print(stock)

MMM
AOS
ABT
ABBV
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ANET
AJG
AIZ
T
ATO
ADSK
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BBWI
BAX
BDX
WRB
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
BG
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FITB
FSLR
FE
FIS
FI
FLT
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GEHC
GEN
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST


In [3]:
len(sp500_stocks)

503

In [4]:
import random

# Set the seed for reproducibility
random.seed(45)

# Generate random selection of 20 stocks
random_stocks = random.sample(sp500_stocks, 50)

# Print the randomly selected stocks
for stock in random_stocks:
    print(stock)

CVS
IT
ITW
GLW
ANSS
D
ENPH
AFL
AME
IEX
ATVI
BKR
DAL
SLB
BAC
WRK
DTE
TTWO
PPL
NDAQ
ALGN
ULTA
MAA
AWK
BIIB
ROP
USB
PNC
AVGO
CB
SYY
PHM
PAYC
GIS
AMGN
MET
FOXA
BKNG
ORCL
AAL
VRTX
ALB
COF
ETN
CEG
EL
RTX
PFE
WMB
EMN


## Getting the Data

In [6]:
start_date = date(2020,1,1)
end_date = date(2020, 12, 31)

# Define the ticker list
tickers_list = random_stocks

# Fetch the training data
import yfinance as yf


#train_data = yf.download(tickers_list, start=start_date, end=end_date)['Adj Close']


train_data = 
for ticker in random_stocks:
    try:
        data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
        train_data[ticker] = data
    except Exception as e:
        print(f"Failed to download data for ticker {ticker}: {e}")

# Print the available tickers
#print(train_data.keys())
#print(data.iloc[-1])
# Print first 5 rows of the data
#print(data.head())
#print(data.tail())

[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker CVS: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker IT: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker ITW: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker GLW: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker ANSS: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker D: list indices must be integers or slices, not str
[*********************100%***********************]  1 

ERROR 
1 Failed download:
ERROR ['CEG']: Exception("CEG: Data doesn't exist for startDate = 1577854800, endDate = 1609390800")



Failed to download data for ticker CEG: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker EL: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker RTX: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker PFE: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker WMB: list indices must be integers or slices, not str
[*********************100%***********************]  1 of 1 completed
Failed to download data for ticker EMN: list indices must be integers or slices, not str


In [ ]:
# Download benchmark data - We can also use Buffet's Portfolio as a Benchmark
spy_data = yf.download('SPY', start=start_date, end=end_date)['Adj Close']

## Applying Markowitz

In [ ]:
# Prepare historical price data for assets
#historical_prices = pd.read_csv('historical_prices.csv', index_col=0, parse_dates=True)
historical_prices = train_data


# Calculate expected returns
returns = expected_returns.mean_historical_return(historical_prices)
#Add shrinkage here
#https://pyportfolioopt.readthedocs.io/en/latest/RiskModels.html#pypfopt.risk_models.CovarianceShrinkage
cov_matrix = risk_models.sample_cov(historical_prices)

# Create an instance of EfficientFrontier
#ef = EfficientFrontier(returns, cov_matrix,weight_bounds = (0,0.2))
ef = EfficientFrontier(returns, cov_matrix, weight_bounds = (0,0.2))

# Efficient Frontier with random portfolios

Maximise the Sharpe Ratio. The result is also referred to as the tangency portfolio, as it is the portfolio for which the capital market line is tangent to the efficient frontier.

This is a convex optimization problem after making a certain variable substitution. See Cornuejols and Tutuncu (2006) for more.

Because max_sharpe() makes a variable substitution, additional objectives may not work as intended.

In [ ]:
fig, ax = plt.subplots()
ef_optimize = ef.deepcopy()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)

### Choosing Optimization Strategy (Comment out the other approach you want to use)

### adding gamma for diversification - https://pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html#l2-regularisation
#ef_optimize.add_objective(objective_functions.L2_reg, gamma=1.5)


### adding transaction costs objective function - https://github.com/robertmartin8/PyPortfolioOpt/blob/master/cookbook/3-Advanced-Mean-Variance-Optimisation.ipynb
# Pretend that you started with a default-weight allocation
#initial_weights = np.array([1/len(tickers_list)] * len(tickers_list))
#ef_optimize.add_objective(objective_functions.transaction_cost, w_prev=initial_weights, k=0.001)

#For Min Volatility
#ef_optimize.min_volatility()

#For Max Sharpe - 
#max_sharpe transforms the optimization problem so additional objectives eg. transaction costs, gamma may not work as expected.

ef_optimize.max_sharpe()

# Markowitz with Target Volatility

#target_volatility = 0.2
#ef_optimize.efficient_risk(target_volatility)

# Find the tangency portfolio

ret_tangent, std_tangent, _ = ef_optimize.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Generate random portfolios
n_samples = 100
w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
# Normalize weights to sum up to 1 - to make sure they lie within efficient frontier
w = w / np.sum(w, axis=1, keepdims=True)  
rets = w.dot(ef.expected_returns)
stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
#ax.set_title("Efficient Frontier with random portfolios")
#ax.legend()
ax.get_legend().remove()
plt.tight_layout()
#Saves the output file as a transparent image
plt.savefig("ef_scatter.png", dpi=200, transparent=True)
plt.show()

In [ ]:
# Print Weights - Max Sharpe
cleaned_weights = ef_optimize.clean_weights()
print(cleaned_weights)

## In-Sample Portfolio Performance

In [ ]:
#to know the expected performance of the portfolio with optimal weights w
ef_optimize.portfolio_performance(verbose=True)

## Comparison with Benchmark Chart

In [ ]:
# Create an empty Series to store portfolio returns
portfolio_returns_1 = pd.Series()
portfolio_returns_1.index.name = 'Date'

# Extract unique years
unique_years = historical_prices.index.year.unique()

In [ ]:
# Iterate through each year
for year in unique_years:
    
    # Filter the DataFrame for the current year
    current_year_prices = historical_prices[historical_prices.index.year == year]
    # to get the last day of the previous year prices
    previous_year_prices = historical_prices[historical_prices.index.year == year-1]
    # Concatenate the last day of the previous year with the current year's data
    # (to avoid NaN value for the first day of the year)
    current_year_prices = pd.concat([previous_year_prices.tail(1), current_year_prices])
    
    # Perform operations on the current_year_prices
    current_year_portfolio_returns = (current_year_prices.pct_change().dropna() * cleaned_weights).sum(axis=1)
    
    # Append current returns to the all_returns Series
    portfolio_returns_1 = pd.concat([portfolio_returns_1, current_year_portfolio_returns])

### In sample graph comparison with Benchmark

In [ ]:
# Calculate portfolio and benchmark returns
portfolio_returns = portfolio_returns_1
benchmark_returns = spy_data.pct_change().dropna()

# Calculate portfolio and benchmark values
#portfolio_values = (1 + portfolio_returns).cumprod() * 10000
#benchmark_values = (1 + benchmark_returns).cumprod() * 10000

# Plot the returns
plt.figure(figsize=(10, 6), facecolor='none')
plt.plot(portfolio_returns.cumsum(), label='Portfolio Returns')
plt.plot(benchmark_returns.cumsum(), label='SPY Returns')
plt.legend()
#plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
#plt.title('Portfolio Returns vs SPY Returns')
# Remove background grid lines
plt.grid(visible=False)
plt.savefig('portfolio_vs_spy.png', transparent=True)

## Markowitz with Monthly Rebalancing keeping Original Weights Only

TODO

## Markowitz with Monthly Rebalancing - With Original Weights Revising every 12 months

TODO

# Hierarchical Risk Parity
HRP is a modern portfolio optimization method inspired by machine learning.

The idea is that by examining the hierarchical structure of the market, we can better diversify.

In [ ]:
rets = expected_returns.returns_from_prices(historical_prices)

In [ ]:
hrp = HRPOpt(rets)
hrp.optimize()
weights = hrp.clean_weights()
weights

In [ ]:
pd.Series(weights).plot.pie(figsize=(10, 10));

In [ ]:
hrp.portfolio_performance(verbose=True);

### Plotting

Plotting a dendrogram (tree diagram) based on the hierarchical structure of asset returns

In [ ]:
plotting.plot_dendrogram(hrp); 

## Simulation

#### Data Universe
Training Data start_date = '2020-01-01'

Training Data end_date = '2020-12-31'

Testing Data = 

In [ ]:
print(end_date)

next_day = end_date + timedelta(days=1)

# We invest on the next day using cleaned_weights

print(next_day)

# Now we keep the target weights same for 1 year

next_year = end_date + relativedelta(years=1)

print(next_year)

In [ ]:
test_data = yf.download(tickers_list, start=next_day, end=next_year)['Adj Close']

In [ ]:
testing_years = test_data.index.year.unique()
testing_months = test_data.index.month.unique()

print(testing_years, testing_months)

In [ ]:
# Calculate the discrete allocation of assets based on the optimal weights
latest_prices = test_data.iloc[0]  # Latest prices for the assets - as on 1 Jan 2021
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
old_allocation, leftover = da.lp_portfolio(verbose=True)

Optimal Allocation: {'AAPL': 16, 'AMD': 22, 'BABA': 3, 'GOOG': 23, 'RRC': 177, 'WMT': 14}

Funds remaining: 99.28
AAPL: allocated 0.190, desired 0.200
AMD: allocated 0.214, desired 0.200
BA: allocated 0.000, desired 0.000
BABA: allocated 0.025, desired 0.075
BAC: allocated 0.000, desired 0.000
GE: allocated 0.000, desired 0.000
GM: allocated 0.000, desired 0.000
GOOG: allocated 0.225, desired 0.200
JPM: allocated 0.000, desired 0.000
MU: allocated 0.000, desired 0.000
PFE: allocated 0.000, desired 0.000
RRC: allocated 0.214, desired 0.125
SBUX: allocated 0.000, desired 0.000
T: allocated 0.000, desired 0.000
UAA: allocated 0.000, desired 0.000
WMT: allocated 0.131, desired 0.200
XOM: allocated 0.000, desired 0.000
Allocation has RMSE: 0.031
Allocation: {'AAPL': 16, 'AMD': 22, 'BABA': 3, 'GOOG': 23, 'RRC': 177, 'WMT': 14}
Leftover: 99.2767562866211

In [ ]:
print(cleaned_weights)

In [ ]:
print("Optimal Allocation:", old_allocation)
print("Leftover:", leftover)
print (10000-leftover)

### Now we calculate the portfolio value after one month and effective weights after one month - which is ratio of current assets vs Portfolio Value

In [ ]:
#find first trading day of the next month

current_month_prices = test_data[(test_data.index.year == 2021) & (test_data.index.month == 12)]

current_month_prices = current_month_prices.sort_index()

#pandas series with stocks name as index, prices are values and date as index name
first_trading_day_prices = current_month_prices.iloc[-1]

In [ ]:
#print(first_trading_day_prices)

In [ ]:
print(old_allocation)

In [ ]:
#print(first_trading_day_prices)

In [ ]:
# Current Value of Each stock in allocated portfolio
result = first_trading_day_prices * pd.Series(old_allocation)

print(result)

In [ ]:
new_portfolio_value = result.sum()

In [ ]:
new_portfolio_value

In [ ]:
#comparison with benchmark
spy_test_data = yf.download('SPY', start=next_day, end=next_year)['Adj Close']

In [ ]:
print(spy_data)

10000/355.447

In [ ]:
current_effective_weights = result/new_portfolio_value

In [ ]:
print(current_effective_weights)

In [ ]:
print(cleaned_weights)

In [ ]:
ordered_dict = cleaned_weights
x1 = old_allocation

print(x1)

## Comparison with Benchmark Chart - Test Data 

In [ ]:
# Create an empty Series to store portfolio returns
portfolio_test_returns = pd.Series()
portfolio_test_returns.index.name = 'Date'

# Extract unique years
unique_years = test_data.index.year.unique()

# Iterate through each year
for year in unique_years:
    
    # Filter the DataFrame for the current year
    current_year_prices = test_data[test_data.index.year == year]
    # to get the last day of the previous year prices
    previous_year_prices = test_data[test_data.index.year == year-1]
    # Concatenate the last day of the previous year with the current year's data
    # (to avoid NaN value for the first day of the year)
    current_year_prices = pd.concat([previous_year_prices.tail(1), current_year_prices])
    
    # Perform operations on the current_year_prices
    current_year_portfolio_returns = (current_year_prices.pct_change().dropna() * cleaned_weights).sum(axis=1)
    
    # Append current returns to the all_returns Series
    portfolio_test_returns = pd.concat([portfolio_test_returns, current_year_portfolio_returns])

In [ ]:
# Calculate portfolio and benchmark returns
portfolio_returns = portfolio_test_returns
benchmark_returns = spy_test_data.pct_change().dropna()

# Plot the returns
plt.figure(figsize=(10, 6), facecolor='none')
plt.plot(portfolio_returns.cumsum(), label='Portfolio Returns')
plt.plot(benchmark_returns.cumsum(), label='SPY Returns')
plt.legend()
#plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
#plt.title('Portfolio Returns vs SPY Returns')
# Remove background grid lines
plt.grid(visible=False)
plt.savefig('portfolio_vs_spy.png', transparent=True)

### working optimization problem

In [ ]:
min_value_constraints = {}
for key, value in x1.items():
    for index, (ordered_key, ordered_value) in enumerate(ordered_dict.items(), start=1):
        if key == ordered_key:
            new_key = int(index-1)
            min_value_constraints[new_key] = value
            break

In [ ]:
min_value_constraints

In [ ]:
total_portfolio_value = 15000

#### Instantiate the CustomDiscreteAllocation class
da = CustomDiscreteAllocation(cleaned_weights, first_trading_day_prices, total_portfolio_value, min_value_constraints=min_value_constraints)

#### Call the lp_portfolio method with the additional constraint
new_allocation, leftover = da.lp_portfolio(reinvest=True, verbose=True)

#### Print the allocation and leftover value
print("Allocation:", new_allocation)
print("Leftover:", leftover)